In [46]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [47]:
import numpy as np
from kalman_estimation import Kalman4FROLS, Selector, get_mat_data
from tqdm import tqdm, trange
from utils import get_term_dict

In [48]:
# !非线性模型
# *非线性数据
terms_path = '../data/linear_terms5D_0.50trial1.mat'
term = Selector(terms_path)
_ = term.make_terms()

# # *保存候选项集合
# # fname = './data/nonlinear_candidate_terms.txt'
# fname = './data/longlag_nonlinear_candidate_terms.txt'
# np.savetxt(fname, terms_repr, fmt='%s')

# *selection
normalized_signals, Kalman_H, candidate_terms, Kalman_S_No = term.make_selection()

# *构造 Kalman Filter
kf = Kalman4FROLS(normalized_signals, Kalman_H=Kalman_H, uc=0.01)
y_coef = kf.estimate_coef()
print(y_coef)

[[ 0.03743781  1.3334661  -0.92166921  0.04183537  0.03789484]
 [ 0.48603659 -0.02310091  0.03083104  0.03444931 -0.01202319]
 [-0.3819153   0.01602031 -0.02170508  0.03269865 -0.00192067]
 [-0.53190126  0.32169573  0.36214341  0.03834424  0.03340586]
 [-0.33967525  0.30404482 -0.03174397  0.03070906 -0.02741767]]


In [49]:
con_terms_linear5 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x1(t-3)', 'x1(t-2)', 'x4(t-1)', 'x5(t-1)', 'x4(t-1)', 'x5(t-1)']  # 9
con_terms_nonlinear5 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)*x1(t-2)', 'x1(t-3)', 'x1(t-2)*x1(t-2)', 'x4(t-1)', 'x5(t-1)', 'x4(t-1)', 'x5(t-1)']  # 9
true_coefs5 = [0.95*np.sqrt(2), -0.9025, 0.5, -0.4, -0.5, 0.25*np.sqrt(2), 0.25*np.sqrt(2), -0.25*np.sqrt(2), 0.25*np.sqrt(2)]  # 9
con_terms_linear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
con_terms_nonlinear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)*x1(t-2)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)*x1(t-2)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
true_coefs10 = [0.95*np.sqrt(2), -0.9025, 0.5, 0.9, -0.5, 0.8, -0.4, -0.8, 0.4, -0.4, -0.9, 0.4, 0.3, -0.3, 0.4, -0.75]  # 16
noises = np.linspace(0.5, 4, 8)
con_terms5 = [2, 1, 1, 3, 2]
con_terms10 = [2, 1, 1, 1, 2, 1, 2, 3, 2, 1]
root = '../data/'

In [50]:
term_dict1, term_dict2 = get_term_dict('nonlinear', 5)

- 形式参数
    - noise_var
    - trial
    - ndim
    - type
    - uc

In [51]:
def corr_term(y_coef, terms_set, Kalman_S_No, var_name: str = 'x', step_name: str = 't'):
    n_dim, n_term = y_coef.shape
    func_repr = []
    for var in range(n_dim):
        y = {}
        for term in range(n_term):
            y[terms_set[Kalman_S_No[var, term]]] = y_coef[var, term]
        func_repr.append(y)
    return func_repr

- frokf 存在一个状态的随机初始化
- 应该多次试验去平均值 ntest设置FROKF进行的实验次数,trials设置的是整体的实验次数

In [52]:
def frokf(noise_var, ndim, dtype, terms, length, root='../data/', trials=100, uc=0.01, ntest=50):
    assert dtype in ['linear', 'nonlinear'], 'type not support!'
    ax = []
    for trial in range(1, trials + 1):
        terms_path = root + f'{dtype}_terms{ndim}D_{noise_var:2.2f}trial{trial}.mat'
        term = Selector(terms_path)
        _ = term.make_terms()
        normalized_signals, Kalman_H, candidate_terms, Kalman_S_No = term.make_selection()
#         Kalman_S_No = np.sort(Kalman_S_No)
        y_coef = 0
        # 对FROKF多次实验取平均值
        for _ in trange(ntest):
            kf = Kalman4FROLS(normalized_signals, Kalman_H=Kalman_H, uc=uc)
            y_coef += kf.estimate_coef()
        y_coef /= ntest
        terms_set = corr_term(y_coef, candidate_terms, Kalman_S_No)
        flatten_coef, t = [], 0
        for i in range(ndim):
            tmp = []
            for k in terms[t:t+length[i]]:
                tmp.append(terms_set[i][k] if k in terms_set[i] else np.nan)
            flatten_coef.extend(tmp)
            t += length[i]
        ax.append(flatten_coef)
    return np.stack(ax)

In [53]:
np.zeros((5, 5))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [54]:
candidate_terms

array(['x1(t-1)', 'x1(t-2)', 'x1(t-3)', 'x1(t-4)', 'x1(t-5)', 'x2(t-1)',
       'x2(t-2)', 'x2(t-3)', 'x2(t-4)', 'x2(t-5)', 'x3(t-1)', 'x3(t-2)',
       'x3(t-3)', 'x3(t-4)', 'x3(t-5)', 'x4(t-1)', 'x4(t-2)', 'x4(t-3)',
       'x4(t-4)', 'x4(t-5)', 'x5(t-1)', 'x5(t-2)', 'x5(t-3)', 'x5(t-4)',
       'x5(t-5)'], dtype='<U7')

In [55]:
term_dict1['x1(t-2)*x1(t-2)']
term_dict2[100]

50

'x1(t-4)*x2(t-2)'

In [57]:
frokf(4, 10, 'nonlinear', con_terms_linear5, con_terms5, uc=1e-6, trials=50)

100%|██████████| 50/50 [04:36<00:00,  5.66s/it]


IndexError: list index out of range

In [ ]:
np.array(con_terms_linear10)

In [ ]:
np.empty((5,5), dtype='<U9')

In [ ]:
uc_map = np.linspace(1e-3, 1e-6, num=8)
uc_map

- 调参